In [ ]:
import os
import shutil
import opendatasets as od
import pandas as pd
from IPython.display import display
from database.sql_connector import SqlConnector

In [ ]:
sql_con = SqlConnector()

# Suicide Victims Data

In [ ]:
kaggle_user = "kritidoneria"
folder_name = "statewise-professional-profile-of-suicide-victims"

In [ ]:
od.download(f"https://www.kaggle.com/{kaggle_user}/{folder_name}", force=True)

In [ ]:
file = os.listdir(folder_name)[0]
df1 = pd.read_csv(os.path.join(folder_name,file), encoding="ISO-8859-1")
df1.columns = ['State', 'Year', 'Cause', 'Male_14', 'Male_15_29', 'Male_30_44', 'Male_45_59', 'Male_60', 'Total_Male',
               'Female_14', 'Female_15_29', 'Female_30_44', 'Female_45_59', 'Female_60', 'Total_Female', 'Total_Male_Female']
df1.sort_values(by=['Year'], ascending=True, inplace=True)
df1_cum = df1.groupby(by=['State', 'Cause']).cumsum().sort_index().drop(columns=['Year'])
df1_cum.insert(0, 'State', df1.State)
df1_cum.insert(1, 'Year', df1.Year)
df1_cum.insert(2, 'Cause', df1.Cause)
df1 = df1_cum

# Remove file 

In [ ]:
shutil.rmtree(folder_name)

# Ingest data 

In [ ]:
sql_con.run_query("""DELETE FROM KAGGLE.DATA.SUICIDE""")
sql_con.ingest_dataframe(df1, schema='DATA', table='SUICIDE')